In [1]:
import pandas as pd
import mlflow

In [4]:
pd.__version__
mlflow.__version__

'2.13.0'

In [5]:
df1 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet')
df2 =  pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet')
df3 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-03.parquet')


In [4]:
df.shape 

(3066766, 19)

In [5]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df2['duration'] = df2.tpep_dropoff_datetime - df2.tpep_pickup_datetime

In [6]:
#df.duration = df.duration.dt

In [7]:
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)



In [8]:
df2.duration = df2.duration.apply(lambda td: td.total_seconds() / 60)


In [9]:
df.duration.std()

42.594351241920904

In [10]:
df = df[(df.duration>=1)&(df.duration <= 60 )]


In [11]:
df2 = df2[(df2.duration>=1)&(df2.duration <= 60 )]

In [12]:
df.shape

(3009173, 20)

In [13]:
 3009173/3066766 

0.9812202822125979

In [14]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333


In [15]:
categorical = ['PULocationID','DOLocationID']


In [16]:
df[categorical] = df[categorical].astype('str')
df2[categorical] = df2[categorical].astype('str')

In [17]:
df[categorical].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3009173 entries, 0 to 3066765
Data columns (total 2 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   PULocationID  object
 1   DOLocationID  object
dtypes: object(2)
memory usage: 68.9+ MB


In [18]:
train_dict = df[categorical].to_dict(orient='records')
val_dict = df2[categorical].to_dict(orient='records')

In [23]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [24]:
target = 'duration'
dv = DictVectorizer()
x_train = dv.fit_transform(train_dict)
y_train = df[target].values

In [25]:
target = 'duration'
x_val = dv.transform(val_dict)
y_val = df2[target].values

In [26]:
x_train.shape

(3009173, 515)

In [27]:
lr = LinearRegression()
lr.fit(x_train,y_train)
y_predict = lr.predict(x_train)

In [28]:
y_val_predict = lr.predict(x_val)

In [29]:
mean_squared_error(y_train,y_predict,squared = False )

7.6492610279057605

In [30]:
mean_squared_error(y_val,y_val_predict,squared = False )

7.81183265470218